## <center> Data Combining Outline </center>
This outline is created to help put together all functions/scripts that were written to populate the final data frame. I will outline the general logic and each necessary step, as well as create some skeleton code to get started. The ultimate goal is to make the combining file run on machine without supervisino. The outline be updated everytime after each meeting. **Please check out your own branch and work separately**.  

**Note**: the following discussion will be broken down into two parts. Multi sources in long period (MS+LP) and single source in short period (SS+SP). To simplify the problem for now, the main discussion will focus on SS+SP, which will be gradually extended to MS+LP.   

======================================================================================================================
### General Steps:
- [ ]  Sound source matching
- [ ]  Timestamp matching
- [ ]  Delay and significance calculation
- [ ]  Extract coordinates
- [ ]  Populate dataframe

======================================================================================================================
### Current Goal:
Formulate data frame for 1 min recording from 1:05 pm - 1:15pm, on March 25th. Single stationary source.

======================================================================================================================

#### Sound Source Matching

The biggest challenge for data combining is sound source matching. Each array has 4 beams/channels, with each channel corresponding to a different sound source. ODAS library is able to track and separate up to 4 different sources at the same time. Currently, we have seen two main problems: cross talk between channels within an array and reflection. The first problem happens when one sound source appears in multiple channels as a result of long silence or ODAS sound source tracking (SST) algorithm failure. The second problem happens when the sound source is too close to a smooth surface and ODAS is picking up sound from direct path and reflection path, and therefore two channels have the same sound. 

Since each recording raw file is approximatly 5 mins long, there is a high chance that each channel within an array corresponds to more than one sound source. Calculating cross corrleation pairs over the entire 5 mins recording for sound source matching (SSM) will not be reliable. Therefore, to adapt to the volatile nature of ODAS SST algorithm, we will implement local SSM, using windowing method with no overlap. 

**MS+LP**: For 1 min recording starting at 1:05pm, given a window length of 1s and 4 arrays (16 channels). The number of cross correlation calculation is 60s/1s * (16 pick 2).

**SS+SP**: Assume only one stationary source is present and no reflection paths are recorded. Then each array should only have one valid channel corresponding to that source. Thus for 1 min recording starting at 1:05pm, given a window length of 1s and 4 arrays (4 valid channels). The number of necessary cross correlation calculation is 60s/1s * (4 pick 2).

**Challenge**:

In [1]:
import pandas as pd
import glob as glob
import copy 

In [18]:
def source_matching(path, number, curDay):
    df = pd.read_csv(path)
    master_list = []
    start_time = df['Time In Seconds'].iloc[0]
    split = start_time + 0.008
    end_time = df['Time In Seconds'].iloc[-1]
    temp = df.iloc[0]
    temp['X'] = None
    temp['Y'] = None
    temp['Z'] = None
    temp['Microphone Number'] = None
    array_0 = temp 
    array_1 = temp
    array_2 = temp
    array_3 = temp
    counter = 0
    #Assuming only 1 channel for each array
    for row in df.iterrows():
        if row[1]['Time In Seconds'] > split:
            dic = {}
            dic['0'] = array_0['X']
            dic['1'] = array_0['Y']
            dic['2'] = array_0['Z']
            dic['3'] = array_1['X']
            dic['4'] = array_1['Y']
            dic['5'] = array_1['Z']
            dic['6'] = array_2['X']
            dic['7'] = array_2['Y']
            dic['8'] = array_2['Z']
            dic['9'] = array_3['X']
            dic['10'] = array_3['Y']
            dic['11'] = array_3['Z']
            
            if dic['0'] != None  or dic['3'] != None or  dic['6'] != None or dic['9'] != None:
                master_list.append(dic)

            split = row[1]['Time In Seconds'] + 0.008

            filler = copy.copy(row[1])
            filler['X'] = None
            filler['Y'] = None
            filler['Z'] = None
            array_0 = filler
            array_1 = filler
            array_2 = filler
            array_3 = filler

            if len(master_list) >= 500000:
                output = pd.DataFrame(master_list)
                print(counter)
                output.to_csv(path_or_buf= "/Users/brian_wangst/Desktop/mlr/data/Fridays/origdata/Ardel/" + curDay + "hour " + str(number)+ " " + str(counter) + ".csv" )      
                master_list = []
                counter = counter + 1
                
        if row[1]['Microphone Number'] == 0:
            array_0 = row[1]
            
        elif row[1]['Microphone Number'] == 1:
            array_1 = row[1]
            
        elif row[1]['Microphone Number'] == 2:
            array_2 = row[1]

        else:
            array_3 = row[1]

                
    output = pd.DataFrame(master_list)
    #output.to_csv(path_or_buf= "/Users/brian_wangst/Desktop/mlr/data/" + path[path.find("data/") + 5:path.find("hour") - 1] + "_" + str(counter) + ".csv")
    output.to_csv(path_or_buf= "/Users/brian_wangst/Desktop/mlr/data/Fridays/origdata/Ardel/" + curDay + "hour " + str(number)+ " " + str(counter) + ".csv" )      

    master_list = []


In [19]:
fridays = ["/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 05:55:33hour0.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 07:23:07hour1.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 08:24:46hour2.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 09:36:43hour3.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 10:37:35hour4.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 11:37:35hour5.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 12:38:15hour6.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 12:01:02hour0.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 01:01:02hour1.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 02:01:57hour2.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 03:01:57hour3.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 04:01:57hour4.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 05:02:34hour5.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 06:13:51hour6.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 08:59:48hour7.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 11:10:53hour8.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 03:07:59hour9.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 06:46:29hour10.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 07:50:51hour11.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 08:53:55hour12.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 09:53:55hour13.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 12:00:38hour0.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 01:03:41hour1.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 02:03:41hour2.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 03:13:01hour3.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 04:29:44hour4.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 07:29:46hour5.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 08:29:46hour6.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 10:27:44hour7.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 12:05:13hour8.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 01:36:21hour9.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 02:40:40hour10.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 04:15:04hour11.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 05:47:44hour12.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 06:51:12hour13.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 07:51:12hour14.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 08:51:57hour15.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/November 22, 2019 02:25:19hour0.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/November 22, 2019 03:27:34hour1.csv"]
counter = 0
curDay = ""
lastDay = "garbage"
for day in fridays:
    curDay = day[day.find("Fridays/") + 8: day.find(":") - 2]
    print(day)
    if lastDay not in curDay:
        print("Switching Days")
        counter = 0
        lastDay = curDay
        source_matching(day, counter, day[day.find("Fridays/") + 8: day.find(":") - 2])
    else:
        counter = counter + 1
        source_matching(day, counter, day[day.find("Fridays/") + 8: day.find(":") - 2])


# source_matching("/Users/brian_wangst/Desktop/mlr/dataJanuary 08, 2020 11:54:14hour3.csv")

/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 05:55:33hour0.csv
Switching Days


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/

/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 07:23:07hour1.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 08:24:46hour2.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 09:36:43hour3.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 10:37:35hour4.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 11:37:35hour5.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 12:38:15hour6.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 12:01:02hour0.csv
Switching Days
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 01:01:02hour1.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 02:01:57hour2.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 03:01:57hour3.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 04:01:57hour4.csv
/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 05:02:34hour5.csv
/

#### Timestamp Matching

In [ ]:
fridays = ["/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 05:55:33hour0.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 07:23:07hour1.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 08:24:46hour2.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 09:36:43hour3.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 10:37:35hour4.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 11:37:35hour5.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 13, 2019 12:38:15hour6.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 12:01:02hour0.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 01:01:02hour1.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 02:01:57hour2.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 03:01:57hour3.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 04:01:57hour4.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 05:02:34hour5.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 06:13:51hour6.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 08:59:48hour7.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 11:10:53hour8.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 03:07:59hour9.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 06:46:29hour10.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 07:50:51hour11.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 08:53:55hour12.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/September 27, 2019 09:53:55hour13.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 12:00:38hour0.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 01:03:41hour1.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 02:03:41hour2.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 03:13:01hour3.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 04:29:44hour4.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 07:29:46hour5.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 08:29:46hour6.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 10:27:44hour7.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 12:05:13hour8.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 01:36:21hour9.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 02:40:40hour10.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 04:15:04hour11.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 05:47:44hour12.csv","/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 06:51:12hour13.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 07:51:12hour14.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/October 18, 2019 08:51:57hour15.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/November 22, 2019 02:25:19hour0.csv", "/Users/brian_wangst/Desktop/mlr/data/Fridays/November 22, 2019 03:27:34hour1.csv"]

